In [ ]:
%%capture
# ---------------------------------------------------------------------------- #
#                        DETAILS TECHNIQUES DU NOTEBOOK                        #
# ---------------------------------------------------------------------------- #

%load_ext autoreload
%autoreload 2

import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', 'src', "tensorflow")))

: 

# Description

Ce notebook présente une première approche des **Gradients Intégrés** avec *Tensorflow*. Nous y suivons le tutoriel disponible ici : https://www.tensorflow.org/tutorials/interpretability/integrated_gradients?hl=fr

Il ne s'agit ni plus ni moins qu'une réimplémentation de ce tutoriel, avec quelques explorations & remarques personnelles

## Chargement & Entrainement du modèle

In [5]:
from Model import Model

model = Model()

ModuleNotFoundError: No module named 'tensorflow_hub'